<a href="https://colab.research.google.com/github/Papa-Panda/Paper_reading/blob/main/Copy_of_Matrix_Multiplication_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile dev_array.h

#ifndef _DEV_ARRAY_H_
#define _DEV_ARRAY_H_

#include <stdexcept>
#include <algorithm>
#include <cuda_runtime.h>

template <class T>
class dev_array
{
    public:
      explicit dev_array()
        : start_(0),
          end_(0)
      {}

      explicit dev_array(size_t size)
      {
          allocate(size);
      }

      ~dev_array(){
          free();
      }

      void resize(size_t size){
          free();
          allocate(size);
      }

      size_t getSize() const {
          return end_ - start_;
      }

      const T* getData() const {
          return start_;
      }

      T* getData() {
          return start_;
      }

      void set(const T* src, size_t size) {
          size_t min = std::min(size, getSize());
          cudaError_t result = cudaMemcpy(start_, src, min * sizeof(T), cudaMemcpyHostToDevice);
          if (result != cudaSuccess)
          {
              throw std::runtime_error("Failed to copy to Device Memory");
          }
      }

      void get(T* dest, size_t size){
          size_t min = std::min(size, getSize());
          cudaError_t result = cudaMemcpy(dest, start_, min * sizeof(T), cudaMemcpyDeviceToHost);
          if (result != cudaSuccess) {
              throw std::runtime_error("Failed to copy to Host Memory");
          }
      }

    private:
      void allocate(size_t size){
          cudaError_t result = cudaMalloc((void**)&start_, size * sizeof(T));
          if (result != cudaSuccess){
              start_ = end_ = 0;
              throw std::runtime_error("Failed to allocate Device Memory");
          }
      }

      void free(){
          if (start_ != 0){
              cudaFree(start_);
              start_ = end_ = 0;
          }
      }

      T* start_;
      T* end_;
};

#endif


Overwriting dev_array.h


In [ ]:
%%writefile kernel.h

#ifndef KERNEL_CUH_
#define KERNEL_CUH_

void matrixMultiplication(float *A, float *B, float *C, int N);

#endif

Writing kernel.h


In [ ]:
%%writefile kernel.cu

#include <math.h>
#include <iostream>
#include "cuda_runtime.h"
#include "kernel.h"
#include <stdlib.h>

using namespace std;

__global__
void matrixMultiplicationKernel(float *A, float *B, float *C, int N){
    int ROW = blockIdx.y * blockDim.y + threadIdx.y;
    int COL = blockIdx.x * blockDim.x + threadIdx.x;

    float tempSum = 0;

    if (ROW < N && COL < N){
        for(int i = 0; i < N; i++){
            tempSum += A[ROW * N + i] * B[i * N + COL];
        }
    }

    C[ROW * N + COL] = tempSum;
}

void matrixMultiplication(float *A, float *B, float *C, int N){

    dim3 threadsPerBlock(N, N);
    dim3 blocksPerGrid(1, 1);

      if (N*N > 512){
          threadsPerBlock.x = 512;
          threadsPerBlock.y = 512;
          blocksPerGrid.x = ceil(double(N)/double(threadsPerBlock.x));
          blocksPerGrid.y = ceil(double(N)/double(threadsPerBlock.y));
      }

      matrixMultiplicationKernel<<<blocksPerGrid, threadsPerBlock>>>(A, B, C, N);
}

Overwriting kernel.cu


In [ ]:
%%writefile matrixmul.cu
#include <stdio.h>
#include <iostream>
#include <vector>
#include <stdlib.h>
#include <time.h>
#include <cuda_runtime.h>
#include "kernel.h"
#include "kernel.cu"
#include "dev_array.h"
#include <math.h>

int main(){

    int N = 3;
    int SIZE = N*N;

    vector<float> h_A(SIZE);
    vector<float> h_B(SIZE);
    vector<float> h_C(SIZE);

    for (int i=0; i<N; i++){
        for (int j=0; j<N; j++){
            h_A[i*N+j] = sin(i);
            h_B[i*N+j] = cos(j);
        }
    }

    dev_array<float> d_A(SIZE);
    dev_array<float> d_B(SIZE);
    dev_array<float> d_C(SIZE);

    d_A.set(&h_A[0], SIZE);
    d_B.set(&h_B[0], SIZE);

    matrixMultiplication(d_A.getData(), d_B.getData(), d_C.getData(), N);
    cudaDeviceSynchronize();

    d_C.get(&h_C[0], SIZE);
    cudaDeviceSynchronize();

    //Testing purpose to compare with CPU Calculation
    float *cpu_C;
    cpu_C = new float[SIZE];

    float sum;
    for (int row = 0; row < N; row++){
        for (int col = 0; col < N; col++){
            sum = 0.0f;
            for(int n = 0; n < N; n++){
                sum += h_A[row * N + n] * h_B[N * n + col];
            }

            cpu_C[row * N + col] = sum;
        }
    }

    double err = 0;

    for (int ROW = 0; ROW < N; ROW++){
        for (int COL = 0; COL < N; COL++){
            err += cpu_C[ROW * N + COL] - h_C[ROW * N + COL];
        }
    }

    std::cout << "Error: "<< err << std::endl;

    return 0;
}

Overwriting matrixmul.cu


In [ ]:
%%shell

nvcc matrixmul.cu -o matrixmul
./matrixmul

nvprof ./matrixmul

Error: -1.19209e-07
==931== NVPROF is profiling process 931, command: ./matrixmul
Error: -1.19209e-07
==931== Profiling application: ./matrixmul
==931== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   42.46%  3.8720us         1  3.8720us  3.8720us  3.8720us  matrixMultiplicationKernel(float*, float*, float*, int)
                   35.09%  3.2000us         2  1.6000us  1.3440us  1.8560us  [CUDA memcpy HtoD]
                   22.46%  2.0480us         1  2.0480us  2.0480us  2.0480us  [CUDA memcpy DtoH]
      API calls:   99.74%  289.25ms         3  96.417ms  3.2870us  289.24ms  cudaMalloc
                    0.12%  340.67us         1  340.67us  340.67us  340.67us  cuDeviceTotalMem
                    0.06%  165.02us       101  1.6330us     141ns  83.500us  cuDeviceGetAttribute
                    0.05%  142.05us         3  47.349us  4.9200us  121.81us  cudaFree
                    0.01%  40.979us         3  13.659us 